In [10]:
from brian2 import *
%matplotlib inline

#basic neuronal group characteristics
neurons_num = 12 #referring to the number of stuctures and subgroups sizes

v_rest = -70*mV #resting potential
v_th = -50*mV #threshold potential
area = 0.785*umetre**2
Cm = 1*ufarad*cm**-2 * area
v_eK = -90*mV
v_eNa = 50*mV
g_Na = 100*msiemens*cm**-2 * area
g_K = 30*msiemens*cm**-2 * area


#Hodgkin-Huxley model equations
eqs = '''
dv/dt = -(g_Na*(m*m*m)*h*(v-v_eNa) + g_K*(n*n*n*n)*(v-v_eK) + I_stim)/Cm : volt
dn/dt = ((0.01*(10 + v))/(exp((10 + v)/10) - 1))*(1-n) - (0.125*exp(v/80))*n : 1
dm/dt = ((0.1*(25 + v))/(exp((25 + v)/10) - 1))*(1-m) - (4*exp(v/18))*m : 1
dh/dt = 0.07*exp(v/20)*(1-h) - (1/(exp((30 + v)/10) + 1))*h : 1
I_stim : amp

'''

#glutamate
model_eqs_glu = '''
receptors_num : 1
v_e : volt
W : metre**3
Kd : mM
gam : siemens
dL/dt = -1.1*10**-9 : mol (event-driven)
dI_stim_syn/dt = receptors_num*(v - v_e)*gam*((L/W/Kd)/(1 + (L/W/Kd))) : amp (clock-driven)

'''

#gaba
model_eqs_gaba = '''
receptors_num : 1
v_e : volt
W : metre**3
Kd : mM
gam : siemens
dL/dt = -4*10**-9 : mol (event-driven)
dI_stim_syn/dt = receptors_num*(v - v_e)*gam*((L/W/Kd)/(1 + (L/W/Kd))) : amp (clock-driven)

'''

Prototype = NeuronGroup(neurons_num, eqs, threshold='v > v_th')
Prototype.v = v_rest

# Mapping subgroups by cerebellum integration circuit
MC_in = Prototype[0]
DSGo = Prototype[1]
DSNoGo = Prototype[2]
GPi = Prototype[3]
GPe = Prototype[5]
STN = Prototype[6]
Th = Prototype[4]
MC_out = Prototype[11]

Pons = Prototype[7]
CC = Prototype[8]
DN = Prototype[9]

SNc = Prototype[10]
#Glu = Prototype[1, 2, 4, 7, 8, 11]
#GABA = Prototype[3, 4, 5, 9]

#Setting up synapses
#excitatory synapses (glutamate)
S_exc = Synapses(Prototype, Prototype, model = Equations(model_eqs_glu), on_pre='L = 1.1*mM')


S_exc.connect(i=MC_in, j=[1, 2])
S_exc.connect(i=Th, j=MC_out)
#S_exc.connect(i=SNc, j=DSGo)

#cerebellum route
S_exc.connect(i=MC_in, j=Pons)
S_exc.connect(i=Pons, j=CC)
S_exc.connect(i=DN, j=Th)

#synapses parameters
S_exc.receptors_num = 2500
S_exc.v_e = +55*mV
S_exc.gam = 10*psiemens
S_exc.W = 20*nmeter * area
S_exc.Kd = 500*nM


#inhibitory synapses (GABA)
S_inh = Synapses(Prototype, Prototype, model = Equations(model_eqs_gaba), on_pre='L = 4*mM')
                
S_inh.connect(i=DSGo, j=GPi)
S_inh.connect(i=DSNoGo, j=GPe)
S_inh.connect(i=GPe, j=GPi)
S_inh.connect(i=GPi, j=Th)
#S_inh.connect(i=SNc, j=DSNoGo)

#cerebellum route
S_inh.connect(i=CC, j=DN)

#synapses parameters
S_inh.receptors_num = 2500
S_inh.v_e = -65*mV
S_inh.gam = 8*psiemens
S_inh.W = 20*nmeter * area
S_inh.Kd = 128*nM


In [11]:
M = StateMonitor(Prototype, 'v', record=True)

run(5000*ms)

plot(M.t/ns, M.v[4], label='Th')

xlabel('Time (ms)')
ylabel('v')
legend();

BrianObjectException: Original error and traceback:
Traceback (most recent call last):
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/equations/equations.py", line 960, in check_units
    check_dimensions(str(eq.expr), self.dimensions[var] / second.dim,
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/equations/unitcheck.py", line 41, in check_dimensions
    expr_dims = parse_expression_dimensions(expression, variables)
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/parsing/expressions.py", line 326, in parse_expression_dimensions
    left_dim = parse_expression_dimensions(expr.left, variables)
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/parsing/expressions.py", line 326, in parse_expression_dimensions
    left_dim = parse_expression_dimensions(expr.left, variables)
  File "/Users/victoria.stelmakh/opt/anaconda3/lib/python3.8/site-packages/brian2/